The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories:

In [ ]:
%pylab inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from matplotlib.pyplot import imread
# import imageio
from matplotlib import image


from sklearn.metrics import accuracy_score

# import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
os.listdir(path)

In [ ]:
data = pd.read_csv(path+'icml_face_data.csv')

In [ ]:
data.head()

 Define training, validation and test data:

In [ ]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

Now we will be using Image Augmentation techniques om our dataset.

In [ ]:
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

Reshape and scale the images:

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
X_train = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
X_val = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
X_test = test_images.astype('float32')/255

Encoding of the target value:

In [ ]:
y_train = keras.utils.to_categorical(train_image_label)
y_val = keras.utils.to_categorical(val_image_label)
y_test = keras.utils.to_categorical(test_image_label)
#keras.utils.to_categorical(train['label'].values)

In [ ]:
pd.DataFrame(y_train)

# Data Augmentation

In [ ]:
# With data augmentation to prevent overfitting
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

# Now Using CNN Model:

In [ ]:
# define vars
input_reshape = (48, 48, 1)

pool_size = (2, 2)

hidden_num_units = 265
output_num_units = 7

epochs = 10
batch_size = 128

In [ ]:
%time



model = Sequential([

Convolution2D(75,(2,2), activation='relu',input_shape=input_reshape),
MaxPooling2D((2,2)),

Convolution2D(50,(2,2), activation='relu'),
MaxPooling2D((2,2)),

Convolution2D(25,(2,2), activation='relu'),

Flatten(),

Dense(hidden_num_units, 'relu'),

Dense(output_num_units,'softmax'),
 ])


In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

trained_model_conv = model.fit(X_train, y_train, epochs =epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
plt.plot(trained_model_conv.history['loss'],label='Train Loss')
plt.plot(trained_model_conv.history['val_loss'],label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

I have added 2 convolutional layers each followed by an activation and then Dropout technique. 20 ephocs 

In [ ]:
model = Sequential()
#Block-1
model.add(Convolution2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = input_reshape))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Convolution2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.2))

#Block-2
model.add(Convolution2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = input_reshape))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Convolution2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.2))

#Block-3
model.add(Convolution2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = input_reshape))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Convolution2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.2))

#Block-4
model.add(Convolution2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = input_reshape))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Convolution2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.2))

#Block-5
model.add(Flatten())
model.add(Dense(units = 256 , activation = 'relu'))
model.add(Dropout(0.3))

#Block-6
model.add(Dense(units = 7 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Before compiling i will create 3 things using keras.callbacks class:

**1-Checkpoint( Function — ModelCheckpoint() )**

It will monitor the validation loss and will try to minimize the loss using the mode=’min’ property. When the checkpoint is reached it will save the best trained weights. Verbose=1 is just for visualization when the code created checkpoint.Here i am using it’s following parameters:

**file-path:** Path to save the model file.Here i am saving the model file with the name EmotionDetectionModel.h5
**monitor:** Quantity to monitor.Here i am monitoring the validation loss.

**mode:** One of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity.

**save_best_only:** If save_best_only=True, the latest best model according to the quantity monitored will not be overwritten.
**verbose:** int. 0: quiet, 1: update messages.

**2-Early Stopping ( Function — EarlyStopping() )**

This will stop the execution early by checking the following properties.

**monitor:** Quantity to monitor.Here i am monitoring the validation loss.

**min_delta:** Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than **min_delta,** will count as no improvement.Here i have given it 0.

**patience:** Number of epochs with no improvement after which training will be stopped. Here i have given it 3.

**restore_best_weights:** Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.Here i have given it True.

**verbose:** int. 0: quiet, 1: update messages.

**3-Reduce Learning Rate ( Function — ReduceLROnPlateau() )**

Models often benefit from reducing the learning rate by a factor of 2–10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a ‘patience’ number of epochs, the learning rate is reduced. I have used the following properties for this.

**monitor**: To monitor a particular loss. Here i am monitoring the validation loss.

**factor**: Factor by which the learning rate will be reduced. new_lr = lr * factor. Here i am using 0.2 as factor.

**patience**: Number of epochs with no improvement after which learning rate will be reduced.Here i am using 3.

**min_delta**: Threshold for measuring the new optimum, to only focus on significant changes.

**verbose**: int. 0: quiet, 1: update messages.

In [ ]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

trained_model_conv = model.fit(X_train, y_train, epochs =20, batch_size=128,callbacks=callbacks, validation_data=(X_val, y_val))

In [ ]:
plt.plot(trained_model_conv.history['loss'],label='Train Loss')
plt.plot(trained_model_conv.history['val_loss'],label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()